In [1]:
%load_ext tensorboard

In [2]:
from datetime import datetime
import io
import itertools

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
        fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

4423680/4422102 [==============================] - 2s 0us/step


In [4]:
print('shape: ', train_images[0].shape)
print('label: ', train_labels[0], '->', class_names[train_labels[0]])

shape:  (28, 28)
label:  9 -> Ankle boot


In [5]:
# Reshape the image for the Summary API
img = np.reshape(train_images[0], (-1, 28, 28, 1))

In [9]:
# Clear out any prior log data
!rm -rf logs

logdir = 'logs/train_data/' + datetime.now().strftime('%Y%m%d-%H%M%S')
file_writer = tf.summary.create_file_writer(logdir=logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
    tf.summary.image('Training data', data=img, step=0)

In [11]:
%tensorboard --logdir logs/train_data

Reusing TensorBoard on port 6009 (pid 21590), started 0:01:18 ago. (Use '!kill 21590' to kill it.)

In [12]:
with file_writer.as_default():
    images = np.reshape(train_images[0:25], (-1, 28, 28, 1))
    tf.summary.image('25 training data examples', images, max_outputs=25, step=0)

In [13]:
%tensorboard --logdir logs/train_data

Reusing TensorBoard on port 6009 (pid 21590), started 0:03:21 ago. (Use '!kill 21590' to kill it.)

In [14]:
!rm -rf logs/plots

In [15]:
logdir = 'logs/plots/' + datetime.now().strftime('%Y%m%d-%H%M%S')
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
    """Converts the matplotlob plot specified by `figure` to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

In [16]:
def image_grid():
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(10, 10))
    for i in range(25):
        # Start next subplot.
        plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i], cmap=plt.cm.binary)
        
    return figure

In [18]:
# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
    tf.summary.image('Training data', plot_to_image(figure), step=0)

In [19]:
%tensorboard --logdir logs/plots

In [20]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [21]:
def plot_confusion_matrix(cm, class_names):
    """Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
        cm (array, shape=[n, n]): a confusion matrix of integer classes
        class_names (array, shape=[n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = 'while' if cm[i, j] > threshold else 'black'
        plt.text(j, i, cm[i, j], horizontalalignment='center', color=color)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [22]:
!rm -rf logs/images

logdir = 'logs/image/' + datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [23]:
def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)
    
    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)
    
    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image('Confusion matrix', cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(op_epoch_end=log_confusion_matrix)

In [25]:
# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0,
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels)
)

In [28]:
%tensorboard --logdir logs/image

Reusing TensorBoard on port 6011 (pid 21731), started 0:03:30 ago. (Use '!kill 21731' to kill it.)